<a href="https://www.kaggle.com/code/amrmohamedsaber/style-classification?scriptVersionId=186959442" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.applications.vgg16 import VGG16
import keras_tuner as kt
from tensorflow.keras.applications import ResNet50,InceptionV3,DenseNet121,Xception,EfficientNetB1
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.metrics import Precision,Recall,F1Score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.preprocessing import image
import os
from glob import glob
import shutil
import random
from shutil import copyfile


# Create Files

In [ ]:
try:
    ###########################################
    os.mkdir('/kaggle/working/art_styles')
    Styles_Train = os.mkdir('/kaggle/working/art_styles/train')
    Styles_Validate = os.mkdir('/kaggle/working/art_styles/validate')
    Styles_Test = os.mkdir('/kaggle/working/art_styles/test')
    ###########################################
    os.mkdir('/kaggle/working/art_styles/train/Baroque')
    os.mkdir('/kaggle/working/art_styles/train/Realism')
    os.mkdir('/kaggle/working/art_styles/train/Expressionism')
    os.mkdir('/kaggle/working/art_styles/train/Cubism')
    os.mkdir('/kaggle/working/art_styles/train/Japanese_Art')
     ###########################################
    os.mkdir('/kaggle/working/art_styles/validate/Baroque')
    os.mkdir('/kaggle/working/art_styles/validate/Realism')
    os.mkdir('/kaggle/working/art_styles/validate/Expressionism')
    os.mkdir('/kaggle/working/art_styles/validate/Cubism')
    os.mkdir('/kaggle/working/art_styles/validate/Japanese_Art')
    ############################################
    os.mkdir('/kaggle/working/art_styles/test/Baroque')
    os.mkdir('/kaggle/working/art_styles/test/Realism')
    os.mkdir('/kaggle/working/art_styles/test/Expressionism')
    os.mkdir('/kaggle/working/art_styles/test/Cubism')
    os.mkdir('/kaggle/working/art_styles/test/Japanese_Art')
    ############################################
except OSError:
    print('Error failed to make directory')
    

os.mkdir('/kaggle/working/art_styles/train/Art_Nouveau_Modern')
os.mkdir('/kaggle/working/art_styles/validate/Art_Nouveau_Modern')
os.mkdir('/kaggle/working/art_styles/test/Art_Nouveau_Modern')

In [ ]:
os.rmdir('/kaggle/working/art_styles/validate/Realism')
os.rmdir('/kaggle/working/art_styles/train/Realism')
os.rmdir('/kaggle/working/art_styles/test/Realism')

# Define the Pathes

In [ ]:
Cubism_DIR = '/kaggle/input/wikiart/Cubism'
Expressionism_DIR = '/kaggle/input/wikiart/Expressionism'
Japanese_Art_DIR = '/kaggle/input/wikiart-art-movementsstyles/Japanese_Art/Japanese_Art'
Baroque_DIR = '/kaggle/input/wikiart/Baroque'
Art_Nouveau_Modern_DIR = '/kaggle/input/wikiart/Art_Nouveau_Modern'

# Fuction To Splite Data

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def split_data(main_dir, training_dir, validation_dir, test_dir, split_size, max_files=2000, random_seed=42):
    # Validate directories and split size
    if not main_dir or not os.path.exists(main_dir):
        logging.error("Error: main_dir is invalid or does not exist.")
        return

    if not (0 < split_size < 1):
        logging.error("Error: split_size must be between 0 and 1.")
        return
    
    # Ensure destination directories exist
    os.makedirs(training_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get all files in the main directory
    all_files = [file for file in os.listdir(main_dir) if os.path.getsize(os.path.join(main_dir, file)) > 0]

    if not all_files:
        logging.error("Error: main_dir is empty.")
        return
    
    # Limit the total number of files to 1500 if more are available
    if len(all_files) > max_files:
        all_files = all_files[:max_files]  # Restrict to 1500 files
        logging.info(f"Limiting to {max_files} files out of {len(all_files)} total.")

    # Split the data into training and validation and test sets
    train_files, remaining_files = train_test_split(all_files, train_size=split_size, random_state=random_seed)
    validation_files, test_files = train_test_split(remaining_files, test_size=0.5, random_state=random_seed)

    # Copy files to their respective directories
    copy_files(main_dir, training_dir, train_files)
    copy_files(main_dir, validation_dir, validation_files)
    copy_files(main_dir, test_dir, test_files)

    logging.info("Split successful!")
    print("Split successful!")

def copy_files(source_dir, destination_dir, file_list):
    for file in file_list:
        try:
            source_path = os.path.join(source_dir, file)
            destination_path = os.path.join(destination_dir, file)
            shutil.copyfile(source_path, destination_path)
        except Exception as e:
            logging.error(f"Error copying {file}: {e}")

In [ ]:
split_data(Baroque_DIR, "/kaggle/working/art_styles/train/Baroque", "/kaggle/working/art_styles/validate/Baroque","/kaggle/working/art_styles/test/Baroque", 0.8)
split_data(Expressionism_DIR, "/kaggle/working/art_styles/train/Expressionism", "/kaggle/working/art_styles/validate/Expressionism", "/kaggle/working/art_styles/test/Expressionism", 0.8)
split_data(Art_Nouveau_Modern_DIR, "/kaggle/working/art_styles/train/Art_Nouveau_Modern", "/kaggle/working/art_styles/validate/Art_Nouveau_Modern", "/kaggle/working/art_styles/test/Art_Nouveau_Modern", 0.8)
split_data(Japanese_Art_DIR, "/kaggle/working/art_styles/train/Japanese_Art", "/kaggle/working/art_styles/validate/Japanese_Art", "/kaggle/working/art_styles/test/Japanese_Art", 0.8)
split_data(Cubism_DIR, "/kaggle/working/art_styles/train/Cubism", "/kaggle/working/art_styles/validate/Cubism", "/kaggle/working/art_styles/test/Cubism", 0.8)

# Preprosseccing

In [ ]:
train_gen = ImageDataGenerator(
        rescale=1./255.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest'                
                   )


validation_gen =  ImageDataGenerator(
        rescale=1./255.)

test_gen =  ImageDataGenerator(
            rescale=1./255.)
train_generator = train_gen.flow_from_directory(
        '/kaggle/working/art_styles/train',
        target_size=(256, 256),
        batch_size=64,
        class_mode="categorical")
validation_generator = validation_gen.flow_from_directory(
        '/kaggle/working/art_styles/validate',
        target_size=(256, 256),
        batch_size=64,
        class_mode="categorical")

test_generator = test_gen.flow_from_directory(
        '/kaggle/working/art_styles/test',
        target_size=(256, 256),
        batch_size=64,
        shuffle= False,
        class_mode="categorical")

# Model

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks for early stopping and learning rate reduction
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

In [ ]:
result = model.fit(
        train_generator,
        epochs=10,
        workers = 5,
        use_multiprocessing = True,
        validation_data=validation_generator)

In [ ]:
for layer in base_model.layers[-35:]:  # Unfreeze the last 35 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
result = model.fit(
        train_generator,
        epochs=50,
        workers = 5,
        use_multiprocessing = True,
        callbacks=[reduce_lr,early_stop],
        validation_data=validation_generator)

In [ ]:
validation_generator = validation_gen.flow_from_directory(
        '/kaggle/working/art_styles/validate',
        target_size=(256, 256),
        batch_size=64,
        shuffle=False,
        class_mode="categorical")
loss,accuracy = model.evaluate(validation_generator)
print("Accuracy:\n", accuracy)
print("Loss:\n", loss)

y_pred_prob = model.predict(validation_generator)  
y_pred = np.argmax(y_pred_prob, axis=1)       
y_true = test_generator.classes

class_labels = list(test_generator.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_labels)

print("Classification Report:\n", report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def display_image_grid(model, n=9, num_cols=3):
    """
    Display a grid of images with their actual and predicted styles.

    Parameters:
    model (tf.keras.Model): Trained model for predicting image styles.
    n (int): Number of images to display. Default is 9.
    num_cols (int): Number of columns in the grid. Default is 3.
    """
    # Calculate the number of rows needed
    num_rows = (n + num_cols - 1) // num_cols

    # Directory containing test images
    test_images_dir = '/kaggle/working/art_styles/test'

    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(6, 6))

    # Iterate over the number of images to display
    for i in range(n):
        row = i // num_cols
        col = i % num_cols

        # Randomly choose a style and an image within that style
        random_style = random.choice(os.listdir(test_images_dir))
        random_image = random.choice(os.listdir(os.path.join(test_images_dir, random_style)))
        random_image_file = os.path.join(test_images_dir, random_style, random_image)

        # Load and preprocess the image
        test_image = image.load_img(random_image_file, target_size=(256, 256, 3))
        test_image = image.img_to_array(test_image) / 255.
        test_image = np.expand_dims(test_image, axis=0)

        # Predict the artist style
        prediction = model.predict(test_image)
        prediction_probability = np.amax(prediction)
        prediction_idx = np.argmax(prediction)

        # Get the labels from the train generator
        labels = train_generator.class_indices
        labels = {v: k for k, v in labels.items()}

        # Determine if the prediction is correct
        actual_style = random_style.replace('_', ' ')
        predicted_style = labels[prediction_idx].replace('_', ' ')
        is_correct = actual_style == predicted_style
        title_color = 'green' if is_correct else 'red'

        # Create the title with actual and predicted styles
        title = f"Actual Style = {actual_style}\n" \
                f"Predicted Style = {predicted_style}\n" \
                f"Prediction probability = {prediction_probability * 100:.2f}%"

        # Display the image with title
        ax = axes[row, col] if num_rows > 1 else axes[col]
        ax.imshow(plt.imread(random_image_file))
        ax.set_title(title, color=title_color)
        ax.axis('off')

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Example usage:
# display_image_grid(model, n=9, num_cols=3)


In [ ]:
display_image_grid(model, n=4, num_cols=2)

In [ ]:

import tensorflow as tf
from tensorflow import lite
from tensorflow.keras.models import load_model

converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('/kaggle/working/style_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Get the unique labels
unique_labels = list(test_generator.class_indices.keys())

# Write the labels to a text file
with open('labels.txt', 'w') as f:
    for label in unique_labels:
        formatted_label = label.replace('_', ' ')
        f.write(str(formatted_label) + '\n')